In [2]:
import re
import ssl
import json
from urllib.request import urlopen
from bs4 import BeautifulSoup

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

### ЭЛЕМЕНТЫ

In [2]:
# Fetch the URL for each author

link = 'https://elementy.ru/novosti_nauki'

author_urls = []

html = urlopen(link, context=ctx).read()
soup = BeautifulSoup(html, 'html.parser')
tags = soup('div', class_='sublink toggle_body minimize')[1].find_all('a')

for tag in tags:
    author_urls.append('https://elementy.ru' + tag['href'])

In [3]:
# Get the number of pages for each author

auth_pages = []

for url in author_urls:
    html = urlopen(url, context=ctx).read()
    soup = BeautifulSoup(html, 'html.parser')
    try:
        pages = soup('a', href=re.compile('.+?page=.+'))
        num = int(pages[-2].get_text())
    except:
        num = 1
    auth_pages.append((url, num))

In [4]:
# Get the URL and publication date for every single article and group them by author

names, d = [], {}

for item in auth_pages:
    url, pages = item[0], item[1]
    
    article, articles = {}, []
    html = urlopen(url, context=ctx).read()
    soup = BeautifulSoup(html, 'html.parser')
    
    name = re.search(r'(.+?)(?= •)', soup.find('title').get_text(' ', strip=True)).group()
    names.append(name)

    for i in range(int(pages)): # look at each page

        html = urlopen(url + f'?page={i}', context=ctx).read()
        soup = BeautifulSoup(html, 'html.parser')
        tags = soup('div', class_='img_block32')

        for tag in tags:
            href = tag.find('a', class_='nohover', href=re.compile('/novosti_nauki/[0-9]+/.+'))
            date = tag.find('span', class_='date').get_text()
            if len(date) < 6: # for newer texts
                date += '.2023'
            link = 'https://elementy.ru' + href['href']

            article = {'Article URL': link, 'Publication date': date}
            articles.append(article)
                
    d[name] = articles

# Write the data to a JSON file
with open('elementy_authors.json', 'w') as f:
    json.dump(d, f, indent=4)

In [6]:
# Retrieve article texts and metadata for chosen authors

data = []

for name in names:

    print(name)

    with open('elementy_authors.json', 'r') as f:
        js = json.load(f)
        
        for i in range(len(js[name])):
            
            d = {}

            # ignore articles published prior to 2010
            if int(js[name][i]['Publication date'][-4:]) < 2010: continue

            url = js[name][i]['Article URL']
            html = urlopen(url, context=ctx).read()
            soup = BeautifulSoup(html, 'html.parser')
            
            # Metadata
            meta = soup.find('div', class_='mb itemhead newslist')
            title = meta.contents[0].get_text()
            date = meta.find('span', class_='date').get_text()
            topics = ', '.join([topic.get_text() for topic in meta('a')[1:-1]])

            pattern = re.compile('Источник(и?)|См. также(:)?(\s)?')
            
            # Full text
            text = ''
            tags = soup.find('div', class_='memo').contents
            for tag in tags[:-2]: # ignore author's signature
                # ignore illustrations, tables, etc.
                if tag.name == 'p' or tag.name == 'ul':
                    # ignore source materials
                    if tag.get('class') and tag.get('class')[0] == 'small': continue
                    if tag.find('b') and pattern.match(tag.find('b').get_text()): continue
                    
                    text += tag.get_text(' ', strip=True) + ' ' 

            if text:
                d = {'Author': name, 'Title': title, 'Publication date': date, \
                    'Topics': topics, 'Article URL': url, 'Full text': text}
                data.append(d)


# Write the data to a JSON file
with open('elementy_texts.json', 'w') as f:
    json.dump(data, f, indent=4)

Айк Акопян
Валентин Анаников
Ольга Баклицкая-Каменева
Дарья Баранова
Вера Башмакова
Арсений Белосохов
Александр Бердичевский
Александр Березин
Антон Бирюков
Алексей Бондарев
Максим Борисов
Варвара Бусова
Анастасия Вабищевич
Ольга Вахрушева
Варвара Веденина
Александр Венедюхин
Кирилл Власов
Михаил Волович
Эдуард Галоян
Михаил Гарбузов
Алексей Гиляров
Ольга Гилярова
Дмитрий Гиляров
Сергей Глаголев
Михаил Гопко
Евгений Гордеев
Анастасия Горелова
Николай Горностаев
Екатерина Грачева
Владимир Гриньков
Анна Гусева
Дмитрий Дагаев
Ира Демина
Екатерина Диффинэ
Татьяна Долгова
Мария Елифёрова
Юрий Ерин
Анастасия Еськова
Дмитрий Жарков
Андрей Журавлёв
Дмитрий Замолодчиков
Никита Зеленков
Денис Земледельцев
Игорь Иванов
Сергей Измалков
Вячеслав Калинин
Андрей Карпачевский
Анна Каспарсон
Павел Квартальнов
Анастасия Кириллова
Мария Кирсанова
Дмитрий Кирюхин
Галина Клинк
Дмитрий Кнорре
Александр Козловский
Сергей Коленов
Кирилл Колесников
Людмила Колупаева
Юлия Кондратенко
Андрей Коньков
Артём Коржим